In [107]:
import plotly.express as px
import plotly.graph_objects as go

from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc

import pandas as pd
import numpy as np

In [108]:
df = pd.read_csv('https://raw.githubusercontent.com/artist0123/ProjectPSIT/main/master.csv', engine = 'python', sep = ',')

In [109]:
df_country = df.groupby(['Country', 'year']).sum().reset_index()
df_country.replace(0, 1, inplace=True)

In [110]:
code_df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2014_world_gdp_with_codes.csv')
df_country_code = df_country.merge(code_df, left_on='Country', right_on='COUNTRY', how='left')
df_year = df.groupby('year').sum()
df_chocolate = df.groupby(['sex', 'age'], as_index=False).sum()

,sex,age,year,suicides_no,population,suicides/100k pop,HDI for year,gdp_per_capita ($)
0,female,15-24 years,4644960,175437,4245159089,10045.33,541.291,39170527
6,male,15-24 years,4644960,633105,4397787807,31487.36,541.291,39170527
1,female,25-34 years,4644960,208823,4190523226,10614.42,541.291,39170527
7,male,25-34 years,4644960,915089,4247580361,45957.10,541.291,39170527
2,female,35-54 years,4644960,506233,7266872023,13732.15,541.291,39170527
8,male,35-54 years,4644960,1945908,7109016100,55653.87,541.291,39170527
3,female,5-14 years,4612704,16997,4107939076,1065.49,541.291,38759885
9,male,5-14 years,4612704,35267,4290754161,1792.90,541.291,38759885
4,female,55-74 years,4644960,430036,4756740046,16533.52,541.291,39170527
10,male,55-74 years,4644960,1228407,4046505294,58460.68,541.291,39170527


In [111]:
 # Build App
YEARS = [x for x in range(1987, 2015)]
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.layout = html.Div(
    id="root",
    children=[
        html.Div(
            id="header",
            className='jumbotron',
            children=[
                html.H1(children="Data Visualization Dashboard")
            ]),
        dbc.Container(fluid=True,
        children=[#Container
            dbc.Row([#Row
                dbc.Col([#Col1
                        dcc.Slider(
                            id='year-slider',
                            min=min(YEARS),
                            max=max(YEARS),
                            value=min(YEARS),
                        ),
                        dcc.Graph(id='world-choropleth'),
                        dcc.Graph(id='everest-graph'),
                        dcc.Graph(id='chocolatebar-graph'),
                    ],
                    md=8),#Col1
                dbc.Col([#Col2
                        dcc.Dropdown(
                            id='select-dropdown',
                            options=[
                                {'label': 'New York City', 'value': 'NYC'},
                                {'label': 'Montreal', 'value': 'MTL'},
                                {'label': 'San Francisco', 'value': 'SF'},
                                {'label': 'Bangkok', 'value': 'BKK'}
                                ],
                            value='NYC'
                            ),
                    html.Div(id='dd-output-container',
                    className='font-weight-bold'),
                ])#Col2
        ])#Row
    ])#Container
])#Div1

In [112]:
# Define callback to update graph
@app.callback(
    Output(component_id='world-choropleth', component_property='figure'),
    Output(component_id='everest-graph', component_property='figure'),
    Output(component_id='chocolatebar-graph', component_property='figure'),
    Output(component_id='dd-output-container', component_property='children'),
    Input(component_id='year-slider', component_property='value'),
    Input(component_id='select-dropdown', component_property='value')
)
def update_figure(year, pltselect):
    df_choropleth = df_country_code.query('year=='+str(year))
    world_suicide = go.Figure(data=go.Choropleth(
    locations = df_choropleth['CODE'],
    z = df_choropleth['suicides_no'],
    text = df_choropleth['Country'],
    colorscale = 'Oryel',
    autocolorscale=False,
    reversescale=False,
    marker_line_color='darkgray',
    marker_line_width=0.2,
    colorbar_title = 'Suicide Cases',
    ))

    world_suicide.update_layout(
    autosize=True,
    title_text='Global Suicide Cases in ' + str(year),
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
        )
    )
    everest_graph = px.line(df_year, x=df_year.index, y="suicides_no")

    chocolatebar_graph = px.bar(df_chocolate, x='sex', y="suicides_no", color='age',                              barmode='group')

    dd_output = 'You have selected' + str(' '+pltselect)
    return world_suicide, everest_graph, chocolatebar_graph, dd_output

In [113]:
# Run app and display result inline in the notebook
app.run_server(mode='external')

Dash app running on http://127.0.0.1:8050/
